ChainerMNで[MNIST](https://en.wikipedia.org/wiki/MNIST_database)(手書き数字認識)のサンプルを動かします。<br>

まず、ChainerMNのGitHubからソースコードをダウンロードします。

In [1]:
curl -L https://github.com/chainer/chainermn/archive/v1.0.0b2.tar.gz | tar zx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   127    0   127    0     0    181      0 --:--:-- --:--:-- --:--:--   181
  0     0    0 1602k    0     0   770k      0 --:--:--  0:00:02 --:--:-- 1281k


chainermn-1.0.0b2というディレクトリが作成されます。

In [ ]:
ls

MNISTのサンプルはchainermn-1.0.0b2/examples/mnist以下にあります。

In [ ]:
ls chainermn-1.0.0b2/examples/mnist

In [ ]:
cat chainermn-1.0.0b2/examples/mnist/train_mnist.py

Chainerを用いたプログラムの作成方法の詳細については下記を参照してください。<br>
http://docs.chainer.org/en/latest/tutorial/index.html<br>
ChaiinerMNを用いたプログラムの作成方法の詳細については下記を参照してください。<br>
https://chainermn.readthedocs.org/en/latest/tutorial<br>


ひとまず、ここでは、どのようなオプションがあるか確認してみましょう。

In [ ]:
python chainermn-1.0.0b2/examples/mnist/train_mnist.py -h

"-g"オプションでGPUを使った実行をすることができます。<br>
"-e 数値"オプションでepch数を指定することができます。<br>
"-b 数値"オプションでミニバッチサイズを指定することができます。このミニバッチサイズに並列に実行するプロセス数を掛けたものが全体のバッチサイズとなります。<br>

このtrain_mnist.pyをMPIを使って実行します。<br>
MPIを使って並列にプロセスを実行する場合にはmpirun (またはmpiexec)を使用します。<br>
どのようなオプションがあるか確認してみましょう。

In [ ]:
mpirun -h

"-n 数値"オプションで並列に実行するプロセス数を指定することができます。<br>
例えば、AAICは1ノードあたり8GPU搭載しているので、1ノードで実行する場合は"-n 8"とします。
2ノードで実行する場合は16GPU(2ノード×8GPU)使うことになるとで"-n 16"とします。

MPIでPythonプログラムを実行する場合は、<br>
mpirun [MPIのオプション] python [Pythonのプログラム名] [Pythonのプログラムのオプション]<br>
という順で引数を記述します。
ここでは、GPUを8台用いて、Epoch数を20、ミニバッチサイズを32（全体のバッチサイズは256)として実行します。 

インタラクティブノードにはGPUが搭載されていないため、以下のサンプルは、
```
$ ssh gpx001.aaic.hpcc.jp
```
とGPUを搭載した異なる計算ノード上で実行することができます。

In [ ]:
mpirun -np 8 python chainermn-1.0.0b2/examples/mnist/train_mnist.py --epoch 20 --batchsize 100 -g

以下のようなエラーが出力される場合がありますが多くの場合実行に問題ありません。<br>
mpirunのオプションに”--mca mpi_warn_on_fork 0”を追加することで出力を抑制することができます。<br>
詳細は下記のサイトを参照してください。<br>
https://github.com/chainer/chainermn/blob/master/docs/source/tutorial/tips_faqs.rst

```
--------------------------------------------------------------------------
A process has executed an operation involving a call to the
"fork()" system call to create a child process.  Open MPI is currently
operating in a condition that could result in memory corruption or
other system errors; your job may hang, crash, or produce silent
data corruption.  The use of fork() (or system() or other calls that
create child processes) is strongly discouraged.

The process that invoked fork was:

  Local host:          [[25170,1],1] (PID 29290)

If you are *absolutely sure* that your application will successfully
and correctly survive a call to fork(), you may disable this warning
by setting the mpi_warn_on_fork MCA parameter to 0.
--------------------------------------------------------------------------
```

無事実行されると、resultというディレクトリにtrain_mnist.pyの実行結果が出力されます。

In [ ]:
ls result

cg.dotはDOT言語で記述されたネットワーク構造のファイル、logはJSONで記述された実行時間、エポック数、反復回数、精度などを記述したファイルになります。